# Normalização

In [2]:
import pandas as pd
from datetime import datetime
import yfinance as yf
from pandas_datareader import data as pdr

import plotly.graph_objs as go


In [3]:
yf.pdr_override()
start_date = datetime(2022, 6, 1)
end_date = datetime(2024, 6, 1)
acoes = ["PETR4.SA","VALE3.SA"]
df_acoes = []
for acao in acoes:
    df = pdr.get_data_yahoo(acao, start=start_date, end=end_date)
    df.index = pd.to_datetime(df.index) # indice nao vem como datetime
    df.columns += " " + acao
    df_acoes.append(df)


yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed



normalization e uniao



In [4]:

for df_acao,acao in zip(df_acoes,acoes):
    colunas_preco = df_acao.columns[:-1]
    coluna_volume = df_acao.columns[-1]  
    df_acao[colunas_preco] = df_acao[colunas_preco]/df_acao["Open "+acao].iloc[0]
    df_acao[coluna_volume] = df_acao[coluna_volume]/df_acao[coluna_volume].iloc[0]
    

df = pd.concat(df_acoes,axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 499 entries, 2022-06-01 to 2024-05-31
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Open PETR4.SA       499 non-null    float64
 1   High PETR4.SA       499 non-null    float64
 2   Low PETR4.SA        499 non-null    float64
 3   Close PETR4.SA      499 non-null    float64
 4   Adj Close PETR4.SA  499 non-null    float64
 5   Volume PETR4.SA     499 non-null    float64
 6   Open VALE3.SA       499 non-null    float64
 7   High VALE3.SA       499 non-null    float64
 8   Low VALE3.SA        499 non-null    float64
 9   Close VALE3.SA      499 non-null    float64
 10  Adj Close VALE3.SA  499 non-null    float64
 11  Volume VALE3.SA     499 non-null    float64
dtypes: float64(12)
memory usage: 50.7 KB


In [5]:
df.head()

,Open PETR4.SA,High PETR4.SA,Low PETR4.SA,Close PETR4.SA,Adj Close PETR4.SA,Volume PETR4.SA,Open VALE3.SA,High VALE3.SA,Low VALE3.SA,Close VALE3.SA,Adj Close VALE3.SA,Volume VALE3.SA
Date,,,,,,,,,,,,
2022-06-01,1.000000,1.014381,0.998328,1.004013,0.551401,1.000000,1.000000,1.030050,0.997796,1.023785,0.873272,1.000000
2022-06-02,1.002341,1.005017,0.988294,0.995318,0.546626,1.094602,1.044205,1.047453,1.032602,1.043044,0.889700,0.957481
2022-06-03,0.993645,1.018060,0.987625,1.012709,0.556177,1.302972,1.038403,1.042000,1.025177,1.026337,0.875449,0.701930
2022-06-06,1.020067,1.021070,1.000334,1.013378,0.556544,0.845980,1.040028,1.041768,1.027381,1.027381,0.876340,0.568676
2022-06-07,1.009699,1.046823,1.008696,1.025418,0.563156,1.733078,1.023785,1.054531,1.022276,1.051398,0.896826,1.028406


In [6]:
df.to_csv("Bolsa.csv")

In [13]:
precos = df[["Close "+acao for acao in acoes] ]
del df
precos = precos.rename(columns={"Close "+acao: acao[:-3] for acao in acoes})
acoes = list(map(lambda nome: nome.split('.SA')[0],precos))
print(acoes)
precos

['PETR4', 'VALE3']


,PETR4,VALE3
Date,,
2022-06-01,1.004013,1.023785
2022-06-02,0.995318,1.043044
2022-06-03,1.012709,1.026337
2022-06-06,1.013378,1.027381
2022-06-07,1.025418,1.051398
...,...,...
2024-05-24,1.224415,0.755076
2024-05-27,1.237793,0.757629
2024-05-28,1.264214,0.741269


In [14]:
# Create a plotly figure
fig = go.Figure()

for acao in acoes:
    # Add first time series
    fig.add_trace(go.Scatter(x=precos.index, y=precos[acao], mode='lines', name=acao))

# Update layout
fig.update_layout(title='valor relativo das ações', xaxis_title='data', yaxis_title='valor relativo')
# Show plot
fig.show()

# Análise

## Geral

In [ ]:
import plotly.express as px

In [ ]:
precos.describe()

,PETR4,VALE3
count,499.000000,499.000000
mean,1.055188,0.822393
std,0.181785,0.099432
min,0.715190,0.676677
25%,0.890406,0.754476
50%,1.042638,0.787625
75%,1.201199,0.873923
max,1.429047,1.110607


In [ ]:
def box_plot(df):
    df_melted = df.melt( value_vars=acoes, var_name='Ações', value_name='Valor Relativo')
    
    # Create box plots using Plotly Express
    fig = px.box(df_melted, x='Ações', y='Valor Relativo',color="Ações", title='Box Plots de Ações')

    # Show plot
    fig.show()
    del df_melted
box_plot(precos)

## Médias Móveis

In [ ]:
def plot_media_mov(acao, range_dias= (0,-1), window_size= 7):

    precos_range = precos[range_dias[0]: range_dias[1]]
    # Create a plotly figure
    fig = go.Figure()

    #plot valores reais
    fig.add_trace(go.Scatter(x=precos_range.index, y= precos_range[acao], mode="lines+markers", name= 'valor'))
    
    media_movel = precos_range[acao].rolling(window= window_size).mean()
    desvio_movel = precos_range[acao].rolling(window= window_size).std()

    desvio_movel_inferior = media_movel - desvio_movel
    desvio_movel_superior = media_movel + desvio_movel

    # Add first time series
    fig.add_trace(go.Scatter(x=media_movel.index, y= desvio_movel_superior, mode='lines', name="+std" ))
    fig.add_trace(go.Scatter(x=media_movel.index, y= desvio_movel_inferior, mode='lines', name="-std" ))
    fig.add_trace(go.Scatter(
        x= precos_range.index.tolist() + precos_range.index.tolist()[::-1],
        y= desvio_movel_inferior.tolist() + desvio_movel_superior[::-1].tolist(),
        fill='toself',
        fillcolor='rgba(0,100,200,0.2)',
        line_color='rgba(255,255,255,0)',
        showlegend=False,
    ))
    
    # Update layout
    fig.update_layout(title= f'valor e desvios da média dos últimos {window_size} dias da ' + acao, xaxis_title='data', yaxis_title='valor relativo')
    # Show plot
    fig.show()

for acao in acoes:
    plot_media_mov(acao, range_dias= (0, 365), window_size= 30)

## Ciclos

In [ ]:
from numpy.fft import fft
import numpy as np

In [ ]:
precos.head()

,PETR4,VALE3
Date,,
2022-06-01,1.000000,1.000000
2022-06-02,0.991339,1.018812
2022-06-03,1.008661,1.002493
2022-06-06,1.009327,1.003513
2022-06-07,1.021319,1.026972


In [ ]:
def fourier_components(precos_series, acao, range_dias=(0,-1), plot_spectrum= True):
    

    # aplica transformada de fourier nos precos de cada uma das ações "normalizados para o range: 1,-1"
    Fprecos = precos_series[range_dias[0]:range_dias[1]].apply(lambda time_series: np.abs(fft(
                                                            2*(time_series - time_series.min())/(time_series.max() - time_series.min())-1
                                                        )),
                        axis= 0
                        )
    
    # as datas nao fazem mais sentido como indice,
    # pois saimos do dominio do tempo para o dominio da frequencia
    Fprecos.reset_index(inplace=True,drop= True) 
    
    # elimina a primeira componente, que indica um offset estabilidade freq = 0
    # elimina a upper side para remover redundância
    N = len(Fprecos)
    Fprecos = Fprecos[1:N//2]
    
    if plot_spectrum:
        dias = precos_series[range_dias[0]:range_dias[1]].index
        year_month_day_format = '%Y-%m-%d'
        primeiro_dia = dias.min().strftime(year_month_day_format)
        ultimo_dia   = dias.max().strftime(year_month_day_format)
        fig = px.bar(Fprecos, 
                     x= Fprecos.index,
                     y=acao, 
                     title=f"espectro de amplitude dos valores da {acao} entre os dias: {primeiro_dia} e {ultimo_dia}")
        fig.show()

    return Fprecos[acao]

In [ ]:
cinco_maiores_componentes = {}
for acao in acoes:
    # print componentes
    componentes = fourier_components(precos, acao, range_dias= (0,365))
    # pega as 5 maiores componentes
    componentes = componentes.sort_values(ascending= False)
    cinco_maiores_componentes[acao] = componentes[:5].index

print("principais frequência")
pd.DataFrame( cinco_maiores_componentes,columns=acoes)

principais frequência


,PETR4,VALE3
0,1,1
1,3,2
2,4,3
3,2,9
4,6,4
